In [1]:
import pandas as pd
import datetime as dt
import random as rnd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [2]:
elo_df = pd.read_csv(r'data\elo_data_set.csv', parse_dates=[0])
elo_df.head()
teams = []
for i in elo_df['team'].unique(): #constructs list of team names
    teams.append(i)

In [3]:
# separates out played games, and the upcoming week.
history_df = elo_df[elo_df['datetime'] < dt.datetime.today()]
future_df = elo_df[elo_df['datetime'] > dt.datetime.today()]
upcoming_week = history_df['week'].iloc[-1] + 1
future_df = future_df[future_df['week'] == upcoming_week]
future_df = future_df[['datetime', 'team', 'opp', 'team_score', 'opp_score', 'team_elo_before', 'opp_elo_before', 'W/L']]
print(f'Week to predict : {upcoming_week}')

Week to predict : 8


In [4]:
# Games to be predicted
future_df.reset_index(inplace=True)
future_df.drop('index', axis=1, inplace=True)
future_df

,datetime,team,opp,team_score,opp_score,team_elo_before,opp_elo_before,W/L
0,2020-10-29 20:20:00,CAR,ATL,NaN,NaN,1420.351117,1451.007231,NaN
1,2020-11-01 13:00:00,MIA,LAR,NaN,NaN,1434.532342,1603.925264,NaN
2,2020-11-01 13:00:00,BAL,PIT,NaN,NaN,1685.266570,1629.930316,NaN
3,2020-11-01 13:00:00,CIN,TEN,NaN,NaN,1282.024705,1616.337140,NaN
4,2020-11-01 13:00:00,CLE,LVR,NaN,NaN,1424.263489,1447.441514,NaN
5,2020-11-01 13:00:00,BUF,NWE,NaN,NaN,1537.921187,1610.302984,NaN
6,2020-11-01 13:00:00,KAN,NYJ,NaN,NaN,1743.292439,1318.121744,NaN
7,2020-11-01 13:00:00,GNB,MIN,NaN,NaN,1644.620252,1515.378678,NaN
8,2020-11-01 13:00:00,DET,IND,NaN,NaN,1400.851537,1487.687626,NaN
9,2020-11-01 16:05:00,DEN,LAC,NaN,NaN,1456.027121,1427.976243,NaN


In [5]:
# Functions to simulate games. variables get defined in the next cell
def gameSim():
    team_1_score = (rnd.gauss(team_1_mean_points, team_1_points_std) + rnd.gauss(team_2_opp_mean, team_2_opp_std))/2
    team_2_score = (rnd.gauss(team_2_mean_points, team_2_points_std) + rnd.gauss(team_1_opp_mean, team_1_opp_std))/2
    if int(round(team_1_score)) > int(round(team_2_score)):
        return 1
    elif int(round(team_1_score)) < int(round(team_2_score)):
        return -1
    else: return 0
    
def gamesSim(ns):
    games = []
    team1win = 0
    team2win = 0
    tie = 0
    for i in range(ns):
        gm = gameSim()
        games.append(gm)
        if gm == 1:
            team1win +=1 
        elif gm == -1:
            team2win +=1
        else: tie +=1 
    print(f'{team_1} Win ', team1win/(team1win+team2win+tie),'%')
    print(f'{team_2} Win ', team2win/(team1win+team2win+tie),'%')
    print('Tie ', tie/(team1win+team2win+tie), '%')
    print('------------------')

In [7]:

counter = 0
for i in future_df:
    team_1_data = pd.DataFrame()
    team_1_offense = []
    team_1_defense = []
    team_1 = future_df['team'][counter]
    team_1_df = history_df[(history_df['team'] == f'{team_1}') | (history_df['opp'] == f'{team_1}')]
    team_1_df = team_1_df[['datetime', 'team', 'opp', 'team_score', 'opp_score', 'team_elo_before', 'opp_elo_before', ]]
    team_1_df.reset_index(inplace=True)
    team_1_df.drop('index', axis=1, inplace=True)
    team_counter = 0
    for x in team_1_df:
        if team_1_df['team'][team_counter] == f'{team_1}':
            team_1_offense.append(team_1_df['team_score'][team_counter])
            team_1_defense.append(team_1_df['opp_score'][team_counter])
        else:
            team_1_offense.append(team_1_df['opp_score'][team_counter])
            team_1_defense.append(team_1_df['team_score'][team_counter])
        team_counter += 1
    
    team_1_data['offense'] = team_1_offense
    team_1_data['defense'] = team_1_defense
    
    team_2_data = pd.DataFrame()
    team_2_offense = []
    team_2_defense = []
    team_2 = future_df['opp'][counter]
    team_2_df = history_df[(history_df['team'] == f'{team_2}') | (history_df['opp'] == f'{team_2}')]
    team_2_df = team_1_df[['datetime', 'team', 'opp', 'team_score', 'opp_score', 'team_elo_before', 'opp_elo_before', ]]
    team_2_df.reset_index(inplace=True)
    team_2_df.drop('index', axis=1, inplace=True)
    team_counter = 0
    for x in team_1_df:
        if team_2_df['team'][team_counter] == f'{team_2}':
            team_2_offense.append(team_2_df['team_score'][team_counter])
            team_2_defense.append(team_2_df['opp_score'][team_counter])
        else:
            team_2_offense.append(team_2_df['opp_score'][team_counter])
            team_2_defense.append(team_2_df['team_score'][team_counter])
        team_counter += 1
        
    team_2_data['offense'] = team_2_offense
    team_2_data['defense'] = team_2_defense
    
    team_1_mean_points = team_1_data['offense'].mean()
    team_2_mean_points = team_2_data['offense'].mean()
    team_1_points_std = team_1_data['offense'].std()
    team_2_points_std = team_2_data['offense'].std()

    team_1_opp_mean = team_1_data['defense'].mean()
    team_2_opp_mean = team_2_data['defense'].mean()
    team_1_opp_std = team_1_data['defense'].std()
    team_2_opp_std = team_1_data['defense'].std()
    
    gamesSim(50000)

    counter += 1

CAR Win  0.43308 %
ATL Win  0.51702 %
Tie  0.0499 %
------------------
MIA Win  0.72458 %
LAR Win  0.24896 %
Tie  0.02646 %
------------------
BAL Win  0.35492 %
PIT Win  0.60294 %
Tie  0.04214 %
------------------
CIN Win  0.06586 %
TEN Win  0.91762 %
Tie  0.01652 %
------------------
CLE Win  0.59626 %
LVR Win  0.37242 %
Tie  0.03132 %
------------------
BUF Win  0.38706 %
NWE Win  0.57582 %
Tie  0.03712 %
------------------
KAN Win  0.56154 %
NYJ Win  0.38842 %
Tie  0.05004 %
------------------
GNB Win  0.62942 %
MIN Win  0.33354 %
Tie  0.03704 %
------------------
